In [0]:
pip install -U openai

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.6/389.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.10/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-a2519988-1bd2-4fd9-b7a1-19aa4d6b2873
    Can't uninstall 'typing_extensions'. No files were found to uninstall.
  Attempting uninstall: openai
    Found existing installation: openai 0.28.1
    Not uninstalling openai at /databricks/python3

In [0]:
dbutils.library.restartPython()

In [0]:
from openai import OpenAI

In [0]:
#### PERFORM TEXT TRANSLATION
DATABRICKS_TOKEN = DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()
negative_review_classes = ["Poor_Quality", "Poor_Customer_Experience", "Camera_Malfunction", "Product_Not_As_Described", "Shipping_Issues", "Design_Or_Build_Issues",  "Tripod_Issues", "Battery_Life_Issues", "Charging_Issues", "App_Or_Software_Issues","Accessories_Issues", "Incompatibility_Issues", "Missing_Items", "Refund_Or_Return_Issues","Packaging_Issues", "Other_Issues"]
def classify_negative_review(review):
  # How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
  # DATABRICKS_TOKEN = os.environ.get('DATABRICKS_TOKEN')
  # Alternatively in a Databricks notebook you can use this:
  client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="<link-to-the-databricks-model-serving-endpoint>"
  )

  chat_completion = client.chat.completions.create(
    messages=[
    {
      "role": "system",
      "content": f'Classify the given negative review into one of the following categories: {negative_review_classes}. Your response must strictly be of the format: class1|class2|class3. Do not change or modify the class names. Here is the review: '
    },
    {
      "role": "user",
      "content": f"{review}"
    }
    ],
    model="databricks-meta-llama-3-1-70b-instruct",
    max_tokens=600
  )
  return chat_completion.choices[0].message.content

In [0]:
from pyspark.sql.functions import udf, split, col
from pyspark.sql.types import StringType, IntegerType

# Register the translate_review function as a UDF
classify_negative_review_udf = udf(classify_negative_review, StringType())

In [0]:
df_sentiment = spark.read.table("harshit_rai_genai_demo.products.camera_reviews_with_sentiment_analysis")

In [0]:
df_negative = df_sentiment.filter(df_sentiment.rating <= 3)

In [0]:
df_classification = df_negative.select("review_id", "translated_review","rating","sentiment", "summary")

In [0]:
from pyspark.sql.functions import col

df_classification = df_classification.withColumn("issue_type", classify_negative_review_udf(col("summary")))

In [0]:
from pyspark.sql.functions import split, when, col, array_contains
split_col = split(col("issue_type"), "\|")
for cls in negative_review_classes:
    df_classification = df_classification.withColumn(cls, when(array_contains(split_col, cls), 1).otherwise(0))

In [0]:
df_classification.write.mode('overwrite').saveAsTable("harshit_rai_genai_demo.products.issue_type_classification")